# pybela Tutorial 2: Streamer – Bela to python advanced
This notebook is a tutorial for the Streamer class in the pybela python library. You can use the Streamer to stream data from Bela to python or vice versa. 

In this tutorial we will be looking at more advanced features to send data from Bela to python. 

The complete documentation for the pybela library can be found in [https://belaplatform.github.io/pybela/](https://belaplatform.github.io/pybela/).

If you didn't do it in the previous tutorial, copy the `bela-code/python-to-bela` project onto Bela. If your Bela is connected to your laptop, you can run the cell below:

In [ ]:
!rsync -rvL ../bela-code/python-to-bela root@bela.local:Bela/projects

Then you can compile and run the project using either the IDE or by running the following command in the Terminal:
```bash
ssh root@bela.local "make -C Bela stop Bela PROJECT=python-to-bela run" 
```
(Running this on a jupyter notebook will block the cell until the program is stopped on Bela.) You will also need to connect two potentiometers to Bela analog inputs 0 and 1. Instructions on how to do so and some details on the Bela code are given in the notebook `1_Streamer-Bela-to-python-basics.ipynb`.

First, we need to import the pybela library, create a Streamer object and connect to Bela.

In [ ]:
import asyncio
from pybela import Streamer

streamer = Streamer()
streamer.connect()

variables = ["pot1", "pot2"]

### Streaming a fixed number of values
You can can use the method `stream_n_values` to stream a fixed number of values of a variable. 

In [ ]:
n_values = 1000
streaming_buffer = streamer.stream_n_values(
            variables=[var["name"] for var in variables], n_values=n_values)

Since the data buffers received from Bela have a fixed size, unless the number of values `n_values` is a multiple of the data buffers size, the streamer will always return a few more values than asked for.

In [ ]:
for var in variables:
    print(f'Variable: {var["name"]}, buffer length: {var["data_length"]}, number of streamed values: {len(streamer.streaming_buffers_data[var["name"]])}')

### Scheduling streaming sessions
You can schedule a streaming session to start and stop at a specific time using the `schedule_streaming()` method. This method takes the same arguments as `start_streaming()`, but it also takes a `timestamps` and `durations` argument.

In [ ]:
latest_timestamp = streamer.get_latest_timestamp() # get the latest timestamp
sample_rate = streamer.sample_rate # get the sample rate
start_timestamp = latest_timestamp + sample_rate # start streaming 1 second after the latest timestamp
duration = sample_rate # stream for 2 seconds

streamer.schedule_streaming(
    variables=variables,
    timestamps=[start_timestamp, start_timestamp],
    durations=[duration, duration],
    saving_enabled=True)

### On-buffer and on-block callbacks
Up until now, we have been streaming data for a period of time and processed the data once the streaming has finished. However, you can also process the data as it is being received. You can do this by passing a callback function to the `on_buffer` or `on_block` arguments of the `start_streaming()` method. 

The `on_buffer` callback will be called every time a buffer is received from Bela. We will need to define a callback function that takes one argument, the buffer. The Streamer will call that function every time it receives a buffer. You can also pass variables to the callback function by using the `callback_args` argument of the `start_streaming()` method. Let's see an example:

In [ ]:
timestamps = {var: [] for var in variables}
buffers = {var: [] for var in variables}

def callback(buffer, timestamps, buffers):
    print("Buffer received")
    
    _var = buffer["name"]
    timestamps[_var].append(
        buffer["buffer"]["ref_timestamp"])
    buffers[_var].append(buffer["buffer"]["data"])
    
    print(_var, timestamps[_var][-1])

streamer.start_streaming(
    variables, saving_enabled=False, on_buffer_callback=callback, callback_args=(timestamps, buffers))

await asyncio.sleep(2)
streamer.stop_streaming()

Let's now look at the `on_block`callback. We call block to a group of buffers. If you are streaming two variables, `pot1` and `pot2`, a block of buffers will contain a buffer for `pot1` and a buffer for `pot2`. If `pot1` and `pot2` have the same buffer size and they are being streamed at the same rate, `pot1` and `pot2` will be aligned in time. This is useful if you are streaming multiple variables and you want to process them together. 

The `on_block` callback will be called every time a block of buffers is received from Bela. We will need to define a callback function that takes one argument, the block. The Streamer will call that function every time it receives a block of buffers. Let's see an example:

In [ ]:
timestamps = {var: [] for var in variables}
buffers = {var: [] for var in variables}

def callback(block, timestamps, buffers):
    print("Block received")
    
    for buffer in block:
        var = buffer["name"]
        timestamps[var].append(buffer["buffer"]["ref_timestamp"])
        buffers[var].append(buffer["buffer"]["data"])

        print(var, timestamps[var][-1])
        
streamer.start_streaming(
    variables, saving_enabled=False, on_block_callback=callback, callback_args=(timestamps, buffers))
await asyncio.sleep(2)
streamer.stop_streaming()